# Aim

The model intends on blindly generating Shakespearean text. An attempt to tackle Attention Architecture(char level).

In [1]:
!wget https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt -O combined-works-external
!pip install tiktoken

'wget' is not recognized as an internal or external command,
operable program or batch file.


# Constants

In [2]:
dataset = 'combined-works-external'
import torch
import torch_directml
dml = torch_directml.device()
# dml = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from params import params
import Model

dml

device(type='privateuseone', index=0)

# Explore

In [3]:
with open(dataset,'r', encoding='utf-8') as file:
    content = file.read()

print(f'Total Size: {len(content)}')

import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

# Function to encode a string
def encode(string):
    return encoding.encode(string)

# Function to decode a list of tokens
def decode(tokens):
    text = encoding.decode(tokens)
    return text

unique_elements = encode(content)

print(f'Total Unique Elements: {len(unique_elements)}')
print(f'Some Elements: {unique_elements[:10]}')

Total Size: 1115394
Total Unique Elements: 301829
Some Elements: [5451, 47317, 512, 10438, 584, 10570, 904, 4726, 11, 6865]


In [4]:
# # Mappings
#
# char_to_int = {ch:i for i,ch in enumerate(unique_elements)}
# int_to_char = {i:ch for i,ch in enumerate(unique_elements)}
#
# # Encoding & Decoding
#
# encode = lambda string: [char_to_int[ch] for ch in string]
# decode = lambda integer: ''.join([int_to_char[i] for i in integer])
#
# print(f'Encoded: {encode("Why am I even doing char based architecture...")}')
# print(f'Decoded: {decode(encode("Why am I even doing char based architecture..."))}')

In [5]:
print(dml)
print(torch.device)
#
# from datasets import load_dataset
# ds = load_dataset("uonlp/CulturaX",
#                   language="en",
#                   use_auth_token=True)


privateuseone:0
<class 'torch.device'>


In [6]:
data = torch.tensor(encode(content), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([301829]) torch.int64


In [7]:
n = int(0.9*len(data))
train = data[:n]
test = data[n:]

In [8]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    loaded_data = train if split == 'train' else test
    ix = torch.randint(len(loaded_data) - params["context_length"], (params["batch_size"],))
    x = torch.stack([loaded_data[i:i+params["context_length"]] for i in ix])
    y = torch.stack([loaded_data[i+1:i+params["context_length"]+1] for i in ix])
    x, y = x.to(dml), y.to(dml)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(params["eval_iters"])
        for k in range(params["eval_iters"]):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
torch.manual_seed(0000)

In [11]:
model = Model.BigramLanguageModel(len(unique_elements), dml)
m = model.to(dml)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=params["learning_rate"])

39.137285 M parameters


In [12]:
for i in range(params["max_iters"]):

    # every once in a while evaluate the loss on train and val sets
    if i % params["eval_interval"] == 0 or i == params["max_iters"] - 1:
        losses = estimate_loss(m)
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

RuntimeError: Could not allocate tensor with 1854588600 bytes. There is not enough GPU video memory available!

In [ ]:
context = torch.tensor(unique_elements[:100], dtype=torch.long, device = dml).view(1, -1)
print(context)
print(decode(m.generate(context, max_new_tokens=2500)[0].tolist()))

In [ ]:
import pickle

with open('SAVED/SAVED_MODEL.pkl', 'wb') as f:
    pickle.dump(m, f)


In [ ]:
with open('SAVED/SAVED_MODEL.pkl', 'rb') as f:
    temp = pickle.load(f)

equal_weights = all(
    torch.equal(param1, param2)
    for param1, param2 in zip(m.parameters(), temp.parameters())
)

print(equal_weights)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')